In [0]:
!pip install mlflow

In [0]:
with open('tokenfile', 'w') as f:
    f.write(dbutils.secrets.get(scope="tokens", key="dbtoken"))
!databricks configure --host https://adb-6724577987585661.1.azuredatabricks.net/ --token-file tokenfile
!rm -rf tokenfile

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

In [0]:
# Create an MLflow client
client = MlflowClient()

In [0]:
# Get the latest version of the model from the experiment
model_name = "titanic_model"
# model_version = client.search_model_versions(f"name='{model_name}'")[0].version
model_version = client.get_latest_versions(model_name, stages=["Production"])[0].version

In [0]:
model_version

In [0]:
import requests
import json

# Define the API endpoint
url = "https://adb-6724577987585661.1.azuredatabricks.net/api/2.0/serving-endpoints"
token = dbutils.secrets.get(scope="tokens", key="dbtoken")

# Define the headers for the request
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Define the data for the request
data = {
    "name": f"titanic_v{model_version}",
    "config": {
        "served_models": [
            {
                "model_name": model_name,
                "model_version": int(model_version),
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ]
    }
}

In [0]:
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(response.json())